## Обучение ранжирующих моделей

## Установка библиотек

In [ ]:
!pip install sentence-transformers==3.1.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 5.1.0
    Uninstalling sentence-transformers-5.1.0:
      Successfully uninstalled sentence-transformers-5.1.0


In [ ]:
!pip install transformers==4.45.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 46.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 23.2 MB/s eta 0:00:00


In [ ]:
!pip install datasets

In [ ]:
import pickle
from random import shuffle

from datasets import load_dataset
from sentence_transformers import InputExample, evaluation

In [ ]:
import torch

device = 'mps' if torch.backends.mps.is_built() else 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Инициализация модели

In [ ]:
from sentence_transformers import SentenceTransformer, models

def raw_bi_encoder():
    word_embedding_model = models.Transformer('cointegrated/rubert-tiny2', max_seq_length=256)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension()) # дополнительно используем слой пулинга
    # для получения векторного представления текста целиком, а не токенов

    bi_encoder = SentenceTransformer(
        modules=[word_embedding_model, pooling_model],
        device=device,
    )

    return bi_encoder

In [ ]:
bi_encoder = raw_bi_encoder()
print(bi_encoder)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 312, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)


## Служебные функции

In [ ]:
# удаление повторяющихся документов
def shrink_repeated_samples(
    queries,
    docs,
    labels,
):
    docs_was = set()
    qs = []
    ds = []
    ls = []
    for i in range(len(queries)):
        q, d, l = queries[i], docs[i], labels[i]
        if d in docs_was:
            continue
        qs.append(q)
        ds.append(d)
        ls.append(l)
        docs_was.add(d)

    return qs, ds, ls

In [ ]:
# загрузка и перемешивание train/val
def trainval_shuffled_data(from_file=False):
    datafile = 'trainval.pkl'

    if from_file:
        with open(datafile, 'rb') as f:
            data = pickle.load(f)
        return data

    good, bad = load_trainval_samples()
    data = good + bad
    shuffle(data)

    with open(datafile, 'wb') as f:
        pickle.dump(data, f)

    return data

In [ ]:
# загрузка и перемешивание test
def test_shuffled_data(from_file=False):
    datafile = 'test.pkl'

    if from_file:
        with open(datafile, 'rb') as f:
            data = pickle.load(f)
        return data

    good, bad = load_test_samples()
    data = good + bad

    with open(datafile, 'wb') as f:
        pickle.dump(data, f)

    return data

In [ ]:
# формирование input example
def trainval_examples():
    data = trainval_shuffled_data(from_file=False)
    # data = data[:75000]

    trainval = []
    for question, document, cos in data:
        trainval.append(
            InputExample(texts=[question, document], label=cos)
        )

    return trainval

In [ ]:
# загрузка тестовых примеров
def load_test_samples():
    _, _, test = _load_sberquad_dataset()

    questions = test['question'].tolist()
    contexts = test['context'].tolist()

    return _inflate_with_negative_samples(
        questions=questions,
        contexts=contexts,
    )

In [ ]:
# загрузка train/val
def load_trainval_samples():
    train, val, _ = _load_sberquad_dataset()

    questions = train['question'].tolist() + val['question'].tolist()
    contexts = train['context'].tolist() + val['context'].tolist()

    return _inflate_with_negative_samples(
        questions=questions,
        contexts=contexts,
    )

In [ ]:
# генерация негативных примеров (негативное семплирование)
def _inflate_with_negative_samples(
    questions,
    contexts,
    delta=30,  # in sberquad dataset same contexts are close
):
    good_q_c_cos = list(zip(questions, contexts, [1.0] * len(questions)))
    bad_q_c_cos = []
    n = len(good_q_c_cos)

    for i in range(n):
        cur_q, cur_c, _ = good_q_c_cos[i]
        next_q, next_c, _ = good_q_c_cos[(i + delta) % n]
        if next_c != cur_c:
            bad_q_c_cos.append((cur_q, next_c, 0.0))

    return good_q_c_cos, bad_q_c_cos

In [ ]:
# загрузка датасета sberquad
def _load_raw_sberquad_dataset(from_file=False):
    sberquad = 'sberquad.pkl'
    if from_file:
        with open(sberquad, 'rb') as f:
            df = pickle.load(f)
    else:
        df = load_dataset('sberquad')
        with open(sberquad, 'wb') as f:
            pickle.dump(df, f)

    return df

In [ ]:
# разбиение на train, validation, test
def _load_sberquad_dataset():
    df = _load_raw_sberquad_dataset()
    train = df['train'].to_pandas()
    validation = df['validation'].to_pandas()
    test = df['test'].to_pandas()

    return train, validation, test

## Метрики близости без дообучения на trainval

In [ ]:
trainval = trainval_shuffled_data(from_file=False)

# n_samples = 1000
# questions, document_contexts, coss = zip(*trainval[:n_samples])
questions, document_contexts, coss = zip(*trainval)

README.md: 0.00B [00:00, ?B/s]

sberquad/train-00000-of-00001.parquet:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

sberquad/validation-00000-of-00001.parqu(…):   0%|          | 0.00/3.43M [00:00<?, ?B/s]

sberquad/test-00000-of-00001.parquet:   0%|          | 0.00/4.93M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

In [ ]:
evaluator = evaluation.EmbeddingSimilarityEvaluator(questions, document_contexts, coss)

In [ ]:
bi_encoder.evaluate(evaluator)

{'pearson_cosine': np.float64(0.769310558201966),
 'spearman_cosine': np.float64(0.7884154971700781),
 'pearson_manhattan': np.float64(0.625522025665652),
 'spearman_manhattan': np.float64(0.6456981414067576),
 'pearson_euclidean': np.float64(0.6211237860402596),
 'spearman_euclidean': np.float64(0.6412993370388377),
 'pearson_dot': np.float64(0.7570034039894602),
 'spearman_dot': np.float64(0.7930446043257426),
 'pearson_max': np.float64(0.769310558201966),
 'spearman_max': np.float64(0.7930446043257426)}

## Метрики близости без дообучения на test

In [ ]:
test = test_shuffled_data(from_file=False)
shuffle(test)

n_test_samples = 1000
test_questions, test_document_contexts, test_coss = zip(*test[:n_test_samples])

In [ ]:
evaluator = evaluation.EmbeddingSimilarityEvaluator(test_questions, test_document_contexts, test_coss)

In [ ]:
bi_encoder = raw_bi_encoder()

In [ ]:
bi_encoder.evaluate(evaluator)

{'pearson_cosine': np.float64(0.7653367694311254),
 'spearman_cosine': np.float64(0.7779718285169896),
 'pearson_manhattan': np.float64(0.6103377231000275),
 'spearman_manhattan': np.float64(0.6257719239466611),
 'pearson_euclidean': np.float64(0.610671376438546),
 'spearman_euclidean': np.float64(0.6246494444537907),
 'pearson_dot': np.float64(0.7654380791654647),
 'spearman_dot': np.float64(0.793783545817795),
 'pearson_max': np.float64(0.7654380791654647),
 'spearman_max': np.float64(0.793783545817795)}

Ожидаемо на необученной модели метрики для обучающих и тестовых данных примерно совпадают

## Дообучение

In [ ]:
# trainval_examples = trainval_examples()[:n_samples]
trainval_examples = trainval_examples()

In [ ]:
print(trainval_examples[0].__dict__)

{'guid': '', 'texts': ['Чье мнение заключалось в том, что некоторые команды используют право доработки двигателя не для надежности а для увеличения мощности?', 'Приём атакующего удара отличается от приёма подачи, так как в защите в обязательном порядке всегда участвуют все 6 игроков, находящихся на площадке; некоторые игроки передней линии ставят блок (иногда все трое), а все остальные играют в защите. Цель защищающихся оставить мяч в игре и по возможности довести его пасующему. Защита может быть эффективной только в случае согласованных действий всех игроков команды, поэтому были разработаны схемы игры в защите, из которых прижились только две: углом назад и углом вперёд . В обеих схемах крайние защитники стоят по боковым линиям, выходя из-за блока в 5-6 метрах от сетки, а вот защитник в 6 зоне, в соответствии с названием схемы, играет или непосредственно позади блока (ловит скидки за блок), или за лицевой линией (играет дальние рикошеты от блока).'], 'label': 0.0}


Негативный семпл

In [ ]:
from torch.utils.data import DataLoader

trainval_dataloader_set = DataLoader(
    trainval_examples,
    shuffle=True,
    batch_size=32,
    collate_fn=bi_encoder.smart_batching_collate # special batch + tokenizer
)

In [ ]:
from sentence_transformers import losses

trainval_loss_set = losses.CosineSimilarityLoss(bi_encoder)

In [ ]:
trainval_loss_set

CosineSimilarityLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 312, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  )
  (loss_fct): MSELoss()
  (cos_score_transformation): Identity()
)

добавили лосс к модели

In [ ]:
# формирование батча
(query_batch, context_batch), labels = next(iter(trainval_dataloader_set))

In [ ]:
query_batch['input_ids'].shape, context_batch['input_ids'].shape, labels.shape

(torch.Size([32, 23]), torch.Size([32, 256]), torch.Size([32]))

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true" # встроен в colab

In [ ]:
# запуск дообучения
bi_encoder.fit(
    train_objectives=[(trainval_dataloader_set, trainval_loss_set)],
    output_path='qa/results',
    epochs=5,
    evaluator=evaluator,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

## Загрузка чекпоинта

In [ ]:
from sentence_transformers import SentenceTransformer

finetuned_bi_encoder = SentenceTransformer('qa/results')

In [ ]:
finetuned_bi_encoder.evaluate(evaluator)

{'pearson_cosine': np.float64(0.879234020346868),
 'spearman_cosine': np.float64(0.8401239337703743),
 'pearson_manhattan': np.float64(0.8760751764812653),
 'spearman_manhattan': np.float64(0.8408791823180464),
 'pearson_euclidean': np.float64(0.8767630475996712),
 'spearman_euclidean': np.float64(0.8409554000613894),
 'pearson_dot': np.float64(0.8573123084617146),
 'spearman_dot': np.float64(0.8317607686599132),
 'pearson_max': np.float64(0.879234020346868),
 'spearman_max': np.float64(0.8409554000613894)}

## Получение предсказаний модели и меток.

Загрузка и отчистка данных

In [ ]:
data = test_shuffled_data(from_file=True)
n_test_samples = 1000
queries, docs, labels = zip(*data[:n_test_samples])

print(f'docs count before shrinking: {len(docs)}')

queries, docs, labels = shrink_repeated_samples(
    queries=queries,
    docs=docs,
    labels=labels,
)

print(f'docs count without duplicates: {len(docs)}')

docs count before shrinking: 1000
docs count without duplicates: 167


Получение эмбеддингов (кодирование)

In [ ]:
doc_embs = finetuned_bi_encoder.encode(
    docs,
    convert_to_tensor=True,
    show_progress_bar=True,
)

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
query_embs = finetuned_bi_encoder.encode(
    queries,
    convert_to_tensor=True,
    show_progress_bar=True,
)

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

## Семантический поиск

In [ ]:
from sentence_transformers import util

cos_scores = util.semantic_search(query_embs, doc_embs, top_k=100)

In [ ]:
print(cos_scores[0][:5])

[{'corpus_id': 0, 'score': 0.8404512405395508}, {'corpus_id': 78, 'score': 0.8367081880569458}, {'corpus_id': 48, 'score': 0.8257060050964355}, {'corpus_id': 35, 'score': 0.8039032220840454}, {'corpus_id': 141, 'score': 0.7665048837661743}]


In [ ]:
top_k = 5
q_idx = 1

In [ ]:
print('Question: ', queries[q_idx])
print()

for no, ir in enumerate(cos_scores[q_idx][:top_k]):
    corpus_id = ir["corpus_id"]
    print('corpus id: ', corpus_id)
    print(f'Document {no + 1}: Cosine Similarity is {ir["score"]:.3f}:\n\n{docs[corpus_id]}')
    print()

Question:  Когда была получена лицензия на осуществление банковских операций в иностранной валюте?

corpus id:  145
Document 1: Cosine Similarity is 0.830:

Банковская система Израиля работает во многих сферах, управляет сберегательными и инвестиционными фондами и выступает в качестве их опекуна, управляет ценными бумагами клиентов, функционирует в качестве их хранителя, дистрибьютор и дилер ценными бумагами на фондовой бирже, выступает в качестве финансового консультанта и т. д. Такая концентрация жизненно важных услуг, и осознание того, что любой сбой в банковской системе нанесет серьёзный ущерб нормальному функционированию всей экономики Израиля, требует от руководства банков предельной ответственности.

corpus id:  166
Document 2: Cosine Similarity is 0.797:

С 1 января 2011 года евро стал в Эстонии единственным законным платёжным средством, наличные кроны имели хождение параллельно с евро в течение 2 недель (в электронных расчётах, очень популярных в Эстонии, переход прошёл одномо

## Расчёт метрик: Recall@5, MRR, MAP, NDCG@10

In [ ]:
def build_target_mask_for_i(i, n):
    target_mask = [False] * n
    target_mask[i] = True
    return target_mask

In [ ]:
def build_pred_mask_for_i(
    cos_scores,
    i,
    n,
):
    pred_mask = [0.0] * n
    qi_scores = cos_scores[i]

    for docid_score in qi_scores:
        doc_id = docid_score['corpus_id']
        score = docid_score['score']
        pred_mask[int(doc_id)] = score

    return pred_mask

### До дообучения

In [ ]:
init_context_embs = bi_encoder.encode(
    queries,
    convert_to_tensor=True,
    show_progress_bar=True,
)

init_question_embs = bi_encoder.encode(
    docs,
    convert_to_tensor=True,
    show_progress_bar=True,
)

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
init_cos_scores = util.semantic_search(init_question_embs, init_context_embs, top_k=100)

In [ ]:
from statistics import mean
from torchmetrics.functional.retrieval import retrieval_reciprocal_rank, retrieval_average_precision, \
    retrieval_normalized_dcg, retrieval_recall

In [ ]:
r5s = []
mrrs = []
rmaps = []
ndcgs = []

for i in range(len(queries)):
    pred = build_pred_mask_for_i(init_cos_scores, i=i, n=len(docs))
    target = build_target_mask_for_i(i=i, n=len(docs))

    pred = torch.tensor(pred)
    target = torch.tensor(target)

    r5 = retrieval_recall(preds=pred, target=target, top_k=5).item()
    mrr = retrieval_reciprocal_rank(preds=pred, target=target, top_k=5).item()
    rmap = retrieval_average_precision(preds=pred, target=target, top_k=5).item()
    ndcg = retrieval_normalized_dcg(preds=pred, target=target, top_k=10).item()

    r5s.append(r5)
    mrrs.append(mrr)
    rmaps.append(rmap)
    ndcgs.append(ndcg)

print(f'mean recall5@{len(docs)}: ', mean(r5s))
print('mean mrr: ', mean(mrrs))
print('mean rmAP: ', mean(rmaps))
print(f'mean ndcg10@{len(docs)}: ', mean(ndcgs))

mean recall5@891:  0.8802395209580839
mean mrr:  0.7874251500575128
mean rmAP:  0.7874251500575128
mean ndcg10@891:  0.8226817610734951


### После дообучения

In [ ]:
r5s = []
mrrs = []
rmaps = []
ndcgs = []

for i in range(len(queries)):
    pred = build_pred_mask_for_i(cos_scores, i=i, n=len(docs))
    target = build_target_mask_for_i(i=i, n=len(docs))

    pred = torch.tensor(pred)
    target = torch.tensor(target)

    r5 = retrieval_recall(preds=pred, target=target, top_k=5).item()
    mrr = retrieval_reciprocal_rank(preds=pred, target=target, top_k=5).item()
    rmap = retrieval_average_precision(preds=pred, target=target, top_k=5).item()
    ndcg = retrieval_normalized_dcg(preds=pred, target=target, top_k=10).item()

    r5s.append(r5)
    mrrs.append(mrr)
    rmaps.append(rmap)
    ndcgs.append(ndcg)

print(f'mean recall5@{len(docs)}: ', mean(r5s))
print('mean mrr: ', mean(mrrs))
print('mean rmAP: ', mean(rmaps))
print(f'mean ndcg10@{len(docs)}: ', mean(ndcgs))

mean recall5@167:  0.8922155688622755
mean mrr:  0.7750499006695376
mean rmAP:  0.7750499006695376
mean ndcg10@167:  0.8236740049487816


# Выводы

**Метрики близости**

|Метрика|Initial|Fine-Tuned|Изменение|
|-------|-------|----------|---------|
|Pearson (cosine)|0.765|0.879|+0.114|
|Spearman (cosine)|0.778|0.840|+0.062|

1. Метрики в целом высокие `~0.85`
2. `Pearson` вырос на `11%`
* Модель стала лучше соотносить абсолютные значения косинусной близости с истинной релевантностью
3. `Spearman` вырос на `6%`
* Модель стала лучше ранжировать релевантные документы
4. **Общий вывод по метрикам близости:**<br>Модель после дообучения научилась более уверенно и последовательно оценивать релевантность контекстов(ответов) к вопросам (запросам) — как в абсолютных значениях близости, так и в порядке их следования.<br>**Fine-tuning дал ощутимый прирост в понимании связей между вопросом и ответом.**

**Метрики ранжирования**

|Метрика|Initial|Fine-Tuned|Изменение|
|-------|-------|----------|---------|
|Recall5@167|0.880|0.892|+0.012|
|MRR|0.787|0.775|-0.012|
|rmAP|0.787|0.775|-0.012|
|NDCG10@167|0.822|0.823|~0|

1. Метрики:
- recall@5 немного улучшился
- ndcg@10 остался почти таким же
- mrr и rmAP немного просели (что может указывать на ухудшение точности ранжирования верхних позиций)
2. Модель, несмотря на свой размер, уже из коробки отлично ранжирует ответы
    - `Recall@5 = 0.88` — уже высокая метрика.
3. Ответы так или иначе отвечают на вопрос. Однако следует вчитываться. Возможно это особенность ответов в датасете `sberquad` т.к. ответы длинные и изначально широкого спектра.